# Data Cleaning, Preparation and Visualization: A Tutorial

- Data Preparation
    - Data loading
    - Data cleaning
    - Data transforming
- Data Visualization

In [ ]:
# import pandas
import pandas as pd
from pandas import Series, DataFrame

## Reading and Writing Data

- pandas features a number of functions for reading tabular data as a DataFrame object.
    - read_csv
    - read_excel
    - read_json
    - read_sql

- DataFrame has methods to write data to a file.
    - to_csv
    - to_json

### Reading Data in Text Format

In [ ]:
# a csv file
!cat students.csv

In [ ]:
# read a csv file
df = pd.read_csv('students.csv')
df

In [ ]:
# specify the index
df2 = pd.read_csv('students.csv', index_col = 'Fname')
df2

In [ ]:
# a csv file without a header row
!cat students2.csv

In [ ]:
# pandas assigns default column names
df3 = pd.read_csv('students2.csv', header = None)
df3

In [ ]:
# specify column names
df4 = pd.read_csv('students2.csv', names = ['Fname', 'Gender', 'Major', 'GPA'])
df4

In [ ]:
# a text file where fields are separated by a variable amount of whitespace
!cat students3.txt

In [ ]:
# read the text file using a regular expression as a delimiter
df5 = pd.read_csv('students3.txt', sep = '\s+')
df5

In [ ]:
# a csv file with missing values
!cat students4.csv

In [ ]:
# read a file with missing values
df6 = pd.read_csv('students4.csv')
df6

### Writing Data to Text Format

In [ ]:
# write the data out to a csv file
df7 = pd.read_csv('students.csv')
# to_csv is a DataFrame's method
df7.to_csv('out.csv', index = False, header = False)
!cat out.csv

In [ ]:
# write a subset of the columns
df7.to_csv('out2.csv', index = False, columns = ['Fname', 'GPA'])
!cat out2.csv

### JSON Data

- JSON has become one of the standard formats for sending data by HTTP request between web browsers and other applications.
- JSON is close to valid Python code. 
- Basic types: objects (dicts), arrays (lists), strings, numbers, booleans, and nulls.
- All of the keys in an JSON object must be strings. 

In [ ]:
# a JSON string
obj = """
[
    {
        "Name": "Mike",
        "Gender": "M", 
        "Major": "FIN",
        "GPA": 3.4
    },
    {
        "Name": "Mary",
        "Gender": "F", 
        "Major": "MGT",
        "GPA": 3.7
    },
    {
        "Name": "Lily",
        "Gender": "F", 
        "Major": null,
        "GPA": 3.2
    }
]
"""

In [ ]:
import json

In [ ]:
# convert a JSON string to a Pyathon form
lst = json.loads(obj)
lst

In [ ]:
# convert a JSON object to a DataFrame
df8 = DataFrame(lst, columns = ['Name', 'Major'])
df8

In [ ]:
# a JSON file
!cat students5.json

In [ ]:
# read a JSON file
df9 = pd.read_json('students5.json')
df9

In [ ]:
# write data to a JSON file
df9.to_json('students6.json', orient = 'records')
!cat students6.json

In [ ]:
# convert a Python object back to JSON
lst2 = df9.to_dict(orient = 'records')
asjson = json.dumps(lst2)
asjson

## Web Scraping

- Web scraping is the practice of automated gathering data from the internet.
- This is accomplished by writing an antomated program that queries a web server, requests data, and then parses the data to extract needed information.
- Web scraping process flow
    - Retrieving HTML data -- **requests** library
    - Parsing the data -- **BeautifulSoup** library and **re** module

### Connecting to Retrieve Data

In [ ]:
import requests

In [ ]:
html = requests.get('http://postmda.github.io/MDA610/Pages/HTMLIntro.html')
# a status code of 200 indicates that the page was downloaded successfully
print(html.status_code)

In [ ]:
# print out the content of the page
print(html.content)

### Parsing HTML Data

- `BeautifulSoup` objects
- `Tag` objects
    - bs.*tagName* returns the first instance of the tag.
    - bs.*find(tagName, tagAttributes)* returns the first instance of the tag with specified attribute values.
    - bs.*find_all(tagName, tagAttributes)* returns a list of the tags.
- `NavigableString` objects
    - tag.*get_text()* returns the text within the tag.

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
# transform HTML into a BeautifulSoup object
bs = BeautifulSoup(html.content, 'html.parser')

In [ ]:
# print out the content of the page with a nice format
print(bs.prettify())

In [ ]:
# print out the title element
print(bs.title)

In [ ]:
# print out the first paragraph
print(bs.p.get_text())

In [ ]:
# print out the first span element with class=attr
print(bs.find('span', {'class': 'attr'}))

In [ ]:
# find all instances of the 'span' tag with class=attr
stag = bs.find_all('span', {'class': 'attr'})
print(stag)
for sp in stag:
    print(sp.get_text())

In [ ]:
# find all instances with the class attribute containing 'text'
pars = bs.find_all(class_='text')
for element in pars:
    print(element.get_text())

In [ ]:
# find an element by id
bs.find(id = 'first').get_text()

### Navigating Trees

- Tree navigation finds a tag based on its location in a document. 
- BeautifulSoup functions always deal with the descendants of the current tag selected.
- tag.*children* returns the elements one level below a parent. 
- *next_sibling, *next_siblings*, *previous_sibling*, and *previous_siblings* can be applied to access the elements at the same level. 

In [ ]:
# descendants at different levels
bs.body.find_all('span')

In [ ]:
# a list of the elements at the top level
lst1 = list(bs.children)
print(lst1)
lst1[0]

### Dow Data Retrieval

In [ ]:
page = requests.get('https://money.cnn.com/data/dow30/')
page.status_code

In [ ]:
# use BeautifulSoup to extract the ticker, name and stock price of each company
bs4 = BeautifulSoup(page.content, 'html.parser')
# a dictionary
dow = {}
# ticker and company name retrieval 
Symbols = bs4.find_all(class_ = 'wsod_firstCol')
dow['Ticker'] = []
dow['Company'] = []
for s in Symbols[1:]:
    txt = s.get_text()
    dow['Ticker'].append(txt[:txt.find(u'\xa0')])
    dow['Company'].append(txt[txt.find(u'\xa0'):].replace(u'\xa0', u''))
# data retrieval
dow['Price'] = []
Elements = bs4.find_all('span', class_ = 'wsod_stream')
print(len(Elements))
i = 0
for item in Elements:
    if i % 3 == 0:
        dow['Price'].append(item.get_text())
    i += 1
DowDF = DataFrame(dow, index = dow['Ticker'])
DowDF

In [ ]:
# use Regular Expressions to extract the ticker, name and stock price of each company
import re
pattern = r'wsod_symbol">(.*?)</a>.*?<span.*?>(.*?)</span>.*?\n.*?wsod_stream">(.*?)</span>'
dlist = re.findall(pattern, page.text)
DowDF2 = DataFrame(dlist, columns = ['Ticker', 'Company', 'Price'])
DowDF2

In [ ]:
# use BeautifulSoup to retrieve all data
bs4 = BeautifulSoup(page.content, 'html.parser')
# a dictionary
dow = {}
# ticker and company name retrieval 
Symbols = bs4.find_all(class_ = 'wsod_firstCol')
dow['Ticker'] = []
dow['Company'] = []
for s in Symbols[1:]:
    txt = s.get_text()
    dow['Ticker'].append(txt[:txt.find(u'\xa0')])
    dow['Company'].append(txt[txt.find(u'\xa0'):].replace(u'\xa0', u''))
# column name retrieval
Header = bs4.find_all('th', {'class': 'wsod_aRight'})
keys = [item.get_text() for item in Header]
for k in keys:
    dow[k] = []
# data retrieval
Data = bs4.find_all('td', class_ = 'wsod_aRight')
i = 0
for s in Data:
    if i % 5 == 0 or i % 5 == 4:
        dow[keys[i % 5]].append(s.find('span').get_text())
    elif i % 5 == 3:
        dow[keys[i % 5]].append(s.get_text())
    else:
        dow[keys[i % 5]].append(s.find('span').find('span').get_text())
    i += 1
dow
dowDF3 = DataFrame(dow, index = dow['Ticker'])
dowDF3

In [ ]:
# use Regular Expressions to extract all data
pattern_header = r'wsod_firstCol">(.*?)</th>\n.*?wsod_aRight">(.*?)</th>\n.*?wsod_aRight">(.*?)</th>\n.*?wsod_aRight">(.*?)</th>\n.*?wsod_aRight">(.*?)</th>\n.*?wsod_aRight">(.*?)</th>'
columns = re.findall(pattern_header, page.text)
print(columns)
clist = list(columns[0])
clist[5] = clist[5].replace('<br>', ' ')
clist.insert(0, 'Ticker')
print(clist)
pattern_data = r'wsod_symbol">(.*?)</a>.*?<span.*?>(.*?)</span>.*?\n.*?wsod_stream">(.*?)</span>.*?\n.*?Data">(.*?)</span>.*?\n.*?ChangePct">(.*?)</span>.*?\n.*?wsod_aRight">(.*?)</td>.*?\n.*?Data">(.*?)</span>'
dlist = re.findall(pattern_data, page.text)
DowDF4 = DataFrame(dlist, columns = clist)
DowDF4

## Handling Missing Data
 
- `float('nan')`, `Numpy.nan` and Python `None` value are treated as missing values in pandas. 
- `isnull()` and `notnull()` are used to detect missing values.
- `dropna()` filters out missing data.
- `fillna()` fills in missing data with some value.

### Filtering Out Missing Data

In [ ]:
# a series
ser = Series([1, 2, np.nan, 5, np.nan])
print(ser)
ser.dropna()

In [ ]:
# a dataframe
df = DataFrame([[1.0, 6.2, 3.0, 7.0], 
                [2.0, np.nan, np.nan, 8.0], 
                [np.nan, 2.0, 3.5, 0.0], 
                [np.nan, 1.0, 4.0, 2.0]], columns = list('abcd'))
print(df)
# dropna() by default drops any rows with missing values
print(df.dropna())
# dropna() drops any column with missing values
df.dropna(axis = 1)

### Filling in Missing Data

In [ ]:
# fill in missing data with a constant
df.fillna(0)

In [ ]:
# use different fill value for each column
df.fillna({'a': 0, 'b': 10, 'c': 100, 'd': 1000})

In [ ]:
# modify the original object in place
df2 = df.copy()
_ = df2.fillna(0, inplace = True)
df2

In [ ]:
# fill in missing values with interpolation: forward fill; backward fill
print(df.fillna(method = 'ffill'))
df.fillna(method = 'bfill')

In [ ]:
# fill in missing values with the mean value
df.fillna(df.mean())

## Data Transformation

### Data Types
- In a DataFrame, each column has its own data type.
- `dtypes` property returns the data types
- `astype()` method changes the data types.

In [ ]:
df2 = DataFrame([[2, 3, 4], [5, 6, 7]], columns = list('abc'))
print(df2.dtypes)
# revise the data type of column 'b'
df2 = df2.astype({'b': np.float64})
df2

In [ ]:
dfa = DataFrame({'A': [1, 2], 'B':[3, 4]})
dfb = DataFrame({'A': [0, 3]})
dfa.add(dfb, fill_value = 0).astype('float64')

### Detecting Outliers

In [ ]:
# any value with a z-score above 3 or below -3 is an outlier
np.random.seed(12345)
df3 = DataFrame(np.random.randn(500, 3))
# number of outliers
(df3.abs() > 3).sum()

In [ ]:
#how many columns have outliers?
print((df3.abs() > 3).any().sum())
# how many rows have outliers?
(df3.abs() > 3).any(axis = 1).sum()

### Replacing Values

- `replace()` method replaces values

In [ ]:
# a single value
ser3 = Series([-1., 0.5, 0.6, -1., 0.2])
print(ser3)
ser3.replace(-1., np.nan)

In [ ]:
# replace a list of values
ser3[3] = -2
ser3.replace([-1, -2], np.nan)

### Encoding a Categorical Variable

- Integer encoding: assign an integer to each distinct category.
- One-hot encoding: if there are `k` categories, then define `k` dummy variables and with `one` in one variable and `0` in the other variables for each category.
- Dummy encoding: if there are `k` categories, then define `k-1` dummy variables and with all zeros for one category, and `one` in one variable and `0` in the other variables for each of the other categories.

In [ ]:
# a Series of ordinal data
ser4 = Series(['Good', 'Poor', 'Good', 'Excellent', 'Excellent'])
ser4

In [ ]:
# integer encoding
ser5 = ser4.replace({'Poor':1, 'Good':2, 'Excellent':3})
ser5

In [ ]:
# a Series of nominal data
ser6 = Series(['English', 'English', 'French', 'French', 'Chinese'])
ser6

In [ ]:
# one-hot encoding with pandas.get_dummies() method
df_e = pd.get_dummies(ser6)
df_e

In [ ]:
# dummy encoding
df_d = df_e.copy()
del df_d['Chinese']
df_d

### Removing Duplicates

In [ ]:
df3 = DataFrame({'a': ['A', 'B'] * 3, 'b': [1, 2, 1] * 2})
df3

In [ ]:
# .duplicated() method detects duplicates
df3.duplicated()

In [ ]:
# .drop_duplicates() method removes duplicates
df3.drop_duplicates()

### Transformation Data Using a Function or Mapping

- `Series.map()` method transforms data in a column or a Series.
- `DataFrame.apply()` method transforms data using a function.

In [ ]:
df4 = DataFrame({'Name': ['Mike', 'Mary', 'Lily'], 
                 'GPA': [2.9, 3.7, 3.5], 
                 'Major': ['MATH', 'FIN', 'EDU']})
df4

In [ ]:
# add a column to the DataFrame
Major_to_College = {'MATH': 'LAS', 'MGT': 'COM', 'FIN': 'COM', 'EDU': 'EDU'}
df4['COL'] = df4.Major.map(Major_to_College)
df4

In [ ]:
df5 = DataFrame(np.random.randn(12).reshape(3, 4), columns = list('abcd'))
# normalization
df5.apply(lambda x: (x.max() - x)/(x.max() - x.min()), axis = 1)

### Discretization and Binning

In [ ]:
# .cut() method
grades = [98, 64, 52, 78, 90, 76, 82, 85, 81]
bins = [50, 60, 70, 80, 90, 100]
cats = pd.cut(grades, bins)
print(cats)
cats.codes

In [ ]:
# frequency distribution
pd.value_counts(cats, sort = False)

## Data Visualization

- Matplotlib is a multiplatform data visualization library built on Numpy arrays.
- In Jupyter notebooks, put all the plotting commands for a figure in a single notebook cell.
- Steps to plot:
    - Create a new figure with `fig = plt.figure()`.
    - Add one or more subplots using `ax = fig.add_subplot()`.
    - Call `ax.plot()` to draw the curve in each subplot.
    - Format the appearance of the axes and lines.

In [ ]:
# interactive plotting in Jupyter notebook
%matplotlib inline
import matplotlib.pyplot as plt

### Figures and Subplots

In [ ]:
# a simple line plot
fig = plt.figure(figsize = (10, 5))   # figsize is in inches
ax = fig.add_subplot(1, 1, 1)
x = np.linspace(0, 10, 100)
ax.plot(x, np.sin(x))

In [ ]:
# multiple plots
fig = plt.figure(figsize = (5, 5))
ax1 = fig.add_subplot(2, 2, 1)
ax2 = fig.add_subplot(2, 2, 2)
ax3 = fig.add_subplot(2, 2, 3)
x = np.linspace(0, 10, 100)
ax1.plot(x, np.sin(x))
ax2.plot(x, np.cos(x))
ax3.plot(x, np.exp(x))

### Colors, Markers and Line Styles

- Colors: 'bgrcmykw'
- Linestyles: '-', '--', '-.', ':'
- Markers: '.O*+xv^<>nph'

In [ ]:
# a line plot with markers
fig = plt.figure(figsize = (10, 5))   
ax = fig.add_subplot(1, 1, 1)
np.random.seed(12345)
x = np.linspace(0, 10, 100)
y = np.random.randn(100)
ax.plot(x, y, 'go--')   # green, circular markers and dashed curve

In [ ]:
# a scatter plot
fig = plt.figure(figsize = (10, 5))   
ax = fig.add_subplot(1, 1, 1)
np.random.seed(12345)
x = np.linspace(0, 5, 10)
y = 2 * x + np.random.randn(10)
ax.plot(x, y, 'ro')   # red, circular markers

In [ ]:
# a histogram
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
data = np.random.randn(1000)
ax.hist(data, bins = 10, density = True, histtype = 'stepfilled', color = 'steelblue', edgecolor = 'none')

### Axes, Title, Labels, and Legends
- Axis limits: `ax.set_xlim()`, `ax.set_ylim()`, or `ax.set_axis()`
- Title: `ax.set_title()`
- Axis labels: `ax.set_xlabel()`, `ax.set_ylabel()`
- Legends: `ax.legend()`

In [ ]:
# a figure with multiple line plots
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
x = np.linspace(0, 10, 100)
ax.plot(x, np.sin(x), label = 'six(x)')
ax.plot(x, np.cos(x), label = 'cos(x)')
ax.set_title('Functions of six(x) and cos(x)')
ax.set_xlabel('x value')
ax.set_ylabel('function')
ax.set_xlim([0, 10])
ax.set_ylim([-1, 1])
ax.legend()